In [1]:
import pandas as pd
import numpy as np
import corenlp
import nltk
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string

<B>The 'Labeled_IL_Reviews.csv' has 1000 samples that are labeled with the aspect terms</B>

In [2]:
df_Labeled = pd.read_csv("./Labeled_IL_Reviews.csv")

In [3]:
df_Labeled.head(3)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tags
0,mofOjB6flg-eAWOFbOkHfQ,0,12-02-2017 20:22,0,8MTptiOpUeuPUFZgtfk9Vw,1,I would have given this restaurant zero stars ...,1,9-vNlkNQL7OJQZ7-U965Ww,"Cleanliness, Food"
1,mofOjB6flg-eAWOFbOkHfQ,0,12/16/2017 4:31,0,ofpfzn8LV4nJ2pE6IlTVdg,1,So...no. Just...no. Before I picked up a plate...,1,#NAME?,"Cleanliness, Food, Service"
2,mofOjB6flg-eAWOFbOkHfQ,0,07-03-2016 20:57,1,O3yApCw95tbA0kodflQrpA,5,The best food I want to the a lot of the buffe...,0,ShcyA_bTtqudK4eaSbtLGA,"Food, Service"


<B>columns like cool, date and funny are not useful for are analysis so we just drop it</B>

In [4]:
df_Labeled = df_Labeled.drop(["cool","date","funny"],axis = 1)

In [5]:
df_Labeled.head(3)

,business_id,review_id,stars,text,useful,user_id,tags
0,mofOjB6flg-eAWOFbOkHfQ,8MTptiOpUeuPUFZgtfk9Vw,1,I would have given this restaurant zero stars ...,1,9-vNlkNQL7OJQZ7-U965Ww,"Cleanliness, Food"
1,mofOjB6flg-eAWOFbOkHfQ,ofpfzn8LV4nJ2pE6IlTVdg,1,So...no. Just...no. Before I picked up a plate...,1,#NAME?,"Cleanliness, Food, Service"
2,mofOjB6flg-eAWOFbOkHfQ,O3yApCw95tbA0kodflQrpA,5,The best food I want to the a lot of the buffe...,0,ShcyA_bTtqudK4eaSbtLGA,"Food, Service"


In [6]:
df_inList = df_Labeled.values.tolist()

<B>A sample of the data is shown below</B>

In [7]:
df_inList[0]

['mofOjB6flg-eAWOFbOkHfQ',
 '8MTptiOpUeuPUFZgtfk9Vw',
 1,
 'I would have given this restaurant zero stars if the app allowed it. This is a disgusting restaurant. You can find roaches lurching around the food. The clean dishes next to buffet items are mostly dirty. Just look at the bottom of the plate. This restaurant should really be inspected by authorities.',
 1,
 '9-vNlkNQL7OJQZ7-U965Ww',
 'Cleanliness, Food']

In [8]:
food_aspects = []
ambience_aspects = []
cleanliness_aspects = []
service_aspects = []

<B>The file Aspect.csv has all the aspect terms that we got from word2vec </B>

In [9]:
df_Aspect = pd.read_csv("Aspect.csv")

In [10]:
service_aspects_word = df_Aspect.Service.to_list()
food_aspects_word = df_Aspect.Food.to_list()
cleanliness_aspects_word = df_Aspect.Cleanliness.to_list()
ambience_aspects_word = df_Aspect.Ambience.to_list()

<B> The method 'addItProperly' adds the review to the given label for example if the parameter label is 'food' then it adds the parameter 'review' to 'food' list

In [11]:
def addItProperly(label,review):
    label = label.lower().strip()
    if label == "food":
        food_aspects.append(review)
        pass
    elif label == "ambience":
        ambience_aspects.append(review)
        pass
    elif label == "cleanliness":
        cleanliness_aspects.append(review)
        pass
    elif label == "service":
        service_aspects.append(review)
        pass

<B>The method 'getLabel' gives the aspect of the input paramter by checking it with Aspect.csv</B>

In [13]:
def getLabel(text):
    temp = nltk.pos_tag(nltk.word_tokenize(text))
    nounList = [val for val in temp if "NN" in val[1]]
    category = ""
    for val in nounList:
        if val[0] in food_aspects_word:
            category = "food"
        elif val[0] in service_aspects_word:
            category = "service"
        elif val[0] in cleanliness_aspects_word:
            category = "cleanliness"
        elif val[0] in ambience_aspects_word:
            category = "ambience"
            pass
    else:
        return category
    return "wrong"

In [14]:
def check_if_present(done,word,s):
    for val in done:
        if val[0] == word:
            return True
    return False

The method 'aspect_based_sentimental_analysis' gives the sentiment analysis for the input parameters it first performs sentence tokenization then It first checks how many aspects does the review belongs to if it has only one aspect then we get the polarity for that review directly. If it has multiple aspect then it first checks the noun term and the adjective and it's dependent adjective and gives it to TextBlob library which gives the polarity for each review.

In [15]:
def aspect_based_sentimental_analysis(df_inList):
    for index, val in enumerate(df_inList):
        Label_Count = len([temp.strip() for temp in val[6].split(",")]) # val[6] is the label count
        if Label_Count == 1:
            sentence_Tokenized = sent_tokenize(val[3])
            one_review_sentiment = []
            for sent_val in sentence_Tokenized:
                s = TextBlob(sent_val).sentiment.polarity
                if s!=0:
                    one_review_sentiment.append(s)
                pass
            if len(one_review_sentiment) !=0 :
                df_inList[index].append(sum(one_review_sentiment)/len(one_review_sentiment))
                pass
            else:
                df_inList[index].append(0)
            pass
            addItProperly([temp.strip() for temp in val[6].split(",")][0],df_inList[index])
        else:
            sentence_Tokenized = sent_tokenize(val[3]) # val[3] gives the text of the data
            for sent_val in sentence_Tokenized:
                done = []
                with corenlp.CoreNLPClient(timeout=30000,annotators="openie".split()) as client:
                    ann = client.annotate(sent_val)
                te = ann.sentence[0]
                for check in te.enhancedDependencies.edge:
                    if "amod" in check.dep or "nsubj" in check.dep:
                        v1 = te.token[check.target - 1].pos
                        v2 = te.token[check.source - 1].pos
                        if ("NN" in v1 and "JJ" in v2) or ("JJ" in v1 and "NN" in v2):
                            print("NN and JJ")
                            se = te.token[check.target - 1].word + " " + te.token[check.source - 1].word
                            s = TextBlob(sent_val).sentiment.polarity
                            print(f"sentence is {sent_val} and polarity is {s}")
                            if (s!= 0) and (not check_if_present(done,v1,s)) and (not check_if_present(done,v2,s)):
                                reviewLabel = getLabel(sent_val)
                                copy = val[:]
                                copy[3] = sent_val
                                copy[6] = reviewLabel
                                copy.append(s)
                                done.append([v1,s])
                                done.append([v2,s])
                                addItProperly(reviewLabel,copy)
                        pass
                    elif check.dep == "asdf":
                        pass
                    pass
                pass
            pass
        pass
    return df_inList

<B> It takes too much time so we have ran it for first five reviews. </B>

In [16]:
temp_df = aspect_based_sentimental_analysis(df_inList[:5])

NN and JJ
sentence is This is a disgusting restaurant. and polarity is -1.0
NN and JJ
sentence is The clean dishes next to buffet items are mostly dirty. and polarity is -0.07777777777777777
NN and JJ
sentence is I alerted a waitress, who acted as if it was nothing new, and she removed the tongs the roach was crawling on. and polarity is 0.13636363636363635
NN and JJ
sentence is The best food I want to the a lot of the buffet restaurant this is the best place to eat and Great service and polarity is 0.9333333333333332
NN and JJ
sentence is Aside from that the bathroom condition was dirty everywhere. and polarity is -0.6
NN and JJ
sentence is The floor was sticky in the main part of the restaurant and especially around where I sat. and polarity is 0.08333333333333333
NN and JJ
sentence is Even the area where the staff hung out, where the drinks were dispensed, was very dirty. and polarity is -0.78
NN and JJ
sentence is The only worthwhile feature is that the food is plentiful. and polar

<B>food_aspects has all the reviews that belonged to food aspect</B>

In [17]:
food_aspects

[['mofOjB6flg-eAWOFbOkHfQ',
  '8MTptiOpUeuPUFZgtfk9Vw',
  1,
  'This is a disgusting restaurant.',
  1,
  '9-vNlkNQL7OJQZ7-U965Ww',
  'food',
  -1.0],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'fFTVZE7EENdl66SX9cgc0A',
  1,
  'The floor was sticky in the main part of the restaurant and especially around where I sat.',
  0,
  'oxcEFkXu_icNyp59Lb8GMQ',
  'food',
  0.08333333333333333],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'pDlIw6DLLjZ5G1vJnMzr9A',
  5,
  'I was visiting the Champaign area with my family who just recently moved there.  And they suggested that I try this buffet.  When I got there was amazed how good their food is.  It is far better than my regular buffet in Lansing.  If I ever return back to Champaign I will be going back, even though it is a little pricy.',
  0,
  'szNYE9rpw4Dv-J0LFs5R7g',
  'Food',
  0.2791666666666666]]

<B>cleanliness_aspects has all the reviews that belonged to cleanliness aspect</B>

In [19]:
cleanliness_aspects

[['mofOjB6flg-eAWOFbOkHfQ',
  '8MTptiOpUeuPUFZgtfk9Vw',
  1,
  'The clean dishes next to buffet items are mostly dirty.',
  1,
  '9-vNlkNQL7OJQZ7-U965Ww',
  'cleanliness',
  -0.07777777777777777],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'ofpfzn8LV4nJ2pE6IlTVdg',
  1,
  'I alerted a waitress, who acted as if it was nothing new, and she removed the tongs the roach was crawling on.',
  1,
  '#NAME?',
  'cleanliness',
  0.13636363636363635],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'fFTVZE7EENdl66SX9cgc0A',
  1,
  'Aside from that the bathroom condition was dirty everywhere.',
  0,
  'oxcEFkXu_icNyp59Lb8GMQ',
  'cleanliness',
  -0.6],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'fFTVZE7EENdl66SX9cgc0A',
  1,
  'I like to eat someplace clean with better food.',
  0,
  'oxcEFkXu_icNyp59Lb8GMQ',
  'cleanliness',
  0.43333333333333335]]

<B>service_aspects has all the reviews that belonged to service aspect</B>

In [20]:
service_aspects

[['mofOjB6flg-eAWOFbOkHfQ',
  'O3yApCw95tbA0kodflQrpA',
  5,
  'The best food I want to the a lot of the buffet restaurant this is the best place to eat and Great service',
  0,
  'ShcyA_bTtqudK4eaSbtLGA',
  'service',
  0.9333333333333332],
 ['mofOjB6flg-eAWOFbOkHfQ',
  'fFTVZE7EENdl66SX9cgc0A',
  1,
  'Even the area where the staff hung out, where the drinks were dispensed, was very dirty.',
  0,
  'oxcEFkXu_icNyp59Lb8GMQ',
  'service',
  -0.78]]

<B> get_rating method converts the sentiment to rating </B>

To get rating from polarity we use the following technique:- <br>
- First we check if the rating is positive then we multiple it by 5. 
- If the rating is less than 0 then we give it one star.

In [25]:
def get_rating(df_list):
    for val in df_list:
#         val[7] is the polarity 
        if val[7] < 0:
            val[7] = 1
        else:
            val[7] = 5 * val[7]
    return df_list

In [26]:
pd.DataFrame(get_rating(food_aspects)).to_csv("./result/food_reviews.csv")

In [27]:
pd.DataFrame(get_rating(service_aspects)).to_csv("./result/service_reviews.csv")

In [28]:
pd.DataFrame(get_rating(cleanliness_aspects)).to_csv("./result/cleanliness_reviews.csv")

In [29]:
pd.DataFrame(get_rating(ambience_aspects)).to_csv("./result/ambience_reviews.csv")

# Conclusion

For checking the work we verified it with restaurant that has business_id as '5-Z7VmiTqD4ycnrxfBJGmQ'.

It has 3 reviews and the rating for them are as follows

<B>1. Decent pizza if you like thin crust. Sometimes great sometimes iffy customer service.</B>
- Here the Food aspect is 0.166.
- The service aspect is 0.8

<B>2. Slow service, they burn the pizza and it is always a complete zoo.  Skip this one and drive to Champaign.</B><br>
This review tells only about the service aspect 
- Service aspect is -0.33

<B>3. This is the pizza place in town that we order from the most.  We are partial to their crispy thin crust pizza.  The food always tastes great just as we expect.  Frequently (at least 50% of the time) they get our order wrong.</B>
- Here the food aspect is 0.8
- The service aspect is -0.4

###### When we calculate the average rating for that restaurant for all the aspects then it comes out to be
- Food - 2.4
- Service - 1.79

### So we can conclude that the restaurant is not overall bad. It just needs to improve it's service aspect to increase customer.

# Future work

As of now, our application tells the rating about the aspects. We aim to make this project to be used in real life for that we require many other features so we plane to include the following features.
- Search options based on cuisine, music, location, etc.
- Select a restaurant based on the signature dish, waiting time, food items.

A user could get answers for the following type of questions like
- Best Chinese restaurant near me
- Which restaurant has the best pizza
- When is the waiting time higher?